In [1]:
# PROJECT/IMPLEMENTATION IDEAS
# 1. Historical market cap, % bitcoin (requires crawling https://coinmarketcap.com/historical/)
# 2. Twitter sentiment analysis of Bitcoin vs price
# 3. Bitcoin vs. altcoin market caps
# 4. Transaction fees, user/address count(?), other features(?) vs bitcoin price
# 5. Usage by country
# 6. Bitcoin trading by exchange
# 7. Bitcoin trading by currency
# 8. Increase in leverage?

In [2]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import patsy
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import RidgeCV
from pandas.core import datetools
%matplotlib inline
import requests
from bs4 import BeautifulSoup
import dateutil.parser
from pprint import pprint
import json
import time
import sys
import re
import datetime

# APIs
import quandl
# import coinmarketcap

/Users/brianmcmahon/anaconda3/envs/tensorflow1.4/lib/python3.5/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [3]:
# Quandl API Calls
df_price = pd.read_csv('https://www.quandl.com/api/v3/datasets/BNC3/GWA_BTC.csv?api_key=pvPBMBW8afR_HqVfio9o') # Price, volume
df_eth = pd.read_csv('https://www.quandl.com/api/v3/datasets/GDAX/ETH_USD.csv?api_key=pvPBMBW8afR_HqVfio9o') # ETH Price, volume
df_fees = pd.read_csv('https://www.quandl.com/api/v3/datasets/BCHAIN/TRFUS.csv?api_key=pvPBMBW8afR_HqVfio9o') # Txn fees
df_cost = pd.read_csv('https://www.quandl.com/api/v3/datasets/BCHAIN/CPTRA.csv?api_key=pvPBMBW8afR_HqVfio9o') # cost per txn
df_no = pd.read_csv('https://www.quandl.com/api/v3/datasets/BCHAIN/NTRAN.csv?api_key=pvPBMBW8afR_HqVfio9o') # num txns
df_noblk = pd.read_csv('https://www.quandl.com/api/v3/datasets/BCHAIN/NTRBL.csv?api_key=pvPBMBW8afR_HqVfio9o') # txns per block
df_blksz = pd.read_csv('https://www.quandl.com/api/v3/datasets/BCHAIN/AVBLS.csv?api_key=pvPBMBW8afR_HqVfio9o') # blk size
df_unq = pd.read_csv('https://www.quandl.com/api/v3/datasets/BCHAIN/NADDU.csv?api_key=pvPBMBW8afR_HqVfio9o') # unique addys
df_hash = pd.read_csv('https://www.quandl.com/api/v3/datasets/BCHAIN/HRATE.csv?api_key=pvPBMBW8afR_HqVfio9o') # hash rate
df_diff = pd.read_csv('https://www.quandl.com/api/v3/datasets/BCHAIN/DIFF.csv?api_key=pvPBMBW8afR_HqVfio9o') # difficulty

df_nasdaq = pd.read_csv('https://www.quandl.com/api/v3/datasets/NASDAQOMX/COMP.csv?api_key=pvPBMBW8afR_HqVfio9o') # NASDAQ Composite
df_nasdaq = df_nasdaq.rename(columns={'Trade Date': 'Date','Index Value':'Nasdaq'})
df_nasdaq = df_nasdaq.drop(['High','Low','Total Market Value','Dividend Market Value'], 1)

df_gold = pd.read_csv('https://www.quandl.com/api/v3/datasets/NASDAQOMX/QGLD.csv?api_key=pvPBMBW8afR_HqVfio9o') # Nasdaq GOLD Index
df_gold = df_gold.rename(columns={'Trade Date': 'Date','Index Value':'Gold'})
df_gold = df_gold.drop(['High','Low','Total Market Value','Dividend Market Value'], 1)

In [4]:
sys.executable

'/Users/brianmcmahon/anaconda3/envs/tensorflow1.4/bin/python'

In [5]:
# Helper functions
def to_currency(int):
    return "${:,.0f}".format(int//1000000)

def to_date(datestring):
    date = dateutil.parser.parse(datestring)
    return date

def list_to_average(list):
    try:
        avg = list[0]/list[1]
    except:
        avg = 0
    return avg

def to_log(num):
    return np.log(num)

# def none_to_value(num):
#     if num == None:
#         num = 


In [6]:
df = df_price.drop('Open', 1)
df = df.drop(['High','Low'], 1)
df = df.rename(columns={'Close': 'BTCPrice','Volume':'BTCVol'})
df = df_eth.merge(df,how='inner',on='Date')
df = df.rename(columns={'Open': 'ETHPrice'})
df = df.drop(['High','Low'], 1)
df = df_fees.merge(df, how='inner', on='Date')
df = df.rename(columns={'Value': 'TxFees'})
df = df_cost.merge(df,how='inner',on='Date')
df = df.rename(columns={'Value': 'CostperTxn'})
df = df_no.merge(df,how='inner',on='Date')
df = df.rename(columns={'Value': 'NoTxns'})
df = df_noblk.merge(df,how='inner',on='Date')
df = df.rename(columns={'Value': 'NoperBlock'})
df = df_blksz.merge(df,how='inner',on='Date')
df = df.rename(columns={'Value': 'AvgBlkSz'})
df = df_unq.merge(df,how='inner',on='Date')
df = df.rename(columns={'Value': 'UniqueAddresses'})
df = df_hash.merge(df,how='inner',on='Date')
df = df.rename(columns={'Value': 'HashRate'})
df = df_diff.merge(df,how='inner',on='Date')
df = df.rename(columns={'Value': 'Difficulty'})

df = df_nasdaq.merge(df,how='inner',on='Date')
df = df_gold.merge(df,how='inner',on='Date')

df['Date'] = df['Date'].apply(to_date)
df['Date'] = pd.to_datetime(df['Date'],format='%Y-%m-%d')

df = df.set_index('Date')

# df['Date'] = pd.to_datetime(str(df['Date']).replace('\n','').replace('-',''),format='%Y%m%d',errors='ignore')
df = df[['BTCPrice','ETHPrice','BTCVol','TxFees','CostperTxn','NoTxns','NoperBlock','AvgBlkSz','UniqueAddresses','HashRate','Difficulty','Nasdaq','Gold']]
# df_hist = df.iloc[250:,:] # fully loaded
df_hist = df

print(df_hist.shape)
df_hist.head(2)

(414, 13)


,BTCPrice,ETHPrice,BTCVol,TxFees,CostperTxn,NoTxns,NoperBlock,AvgBlkSz,UniqueAddresses,HashRate,Difficulty,Nasdaq,Gold
Date,,,,,,,,,,,,,
2018-01-26,11168.196000,1054.03,495536.347799,2.476759e+06,104.088588,236580.0,1497.341772,1.110327,539581.0,2.044515e+07,2.460585e+12,7505.77,185.85
2018-01-25,11352.526793,1033.28,345878.245240,3.198499e+06,111.888553,250423.0,1422.857955,1.069994,576212.0,1.949146e+07,2.227848e+12,7411.16,186.29


In [7]:
df_hist.tail(2)

,BTCPrice,ETHPrice,BTCVol,TxFees,CostperTxn,NoTxns,NoperBlock,AvgBlkSz,UniqueAddresses,HashRate,Difficulty,Nasdaq,Gold
Date,,,,,,,,,,,,,
2016-05-27,486.962703,12.47,762209.731197,25184.016948,7.157532,220093.0,1630.318519,0.823084,364137.0,1.357376e+06,1.993121e+11,4933.50,168.45
2016-05-26,458.279956,12.61,477365.253604,26820.960997,6.389729,247844.0,1795.971014,0.871457,397464.0,1.377192e+06,1.993121e+11,4901.77,172.67


In [8]:
# run coinmarketcap_hist.py weekly to generate .json file
# !python coinmarketcap_hist.py
mkt_cap = pd.read_json('coinmarketcap_hist1.json').T
mkt_cap['Date'] = mkt_cap.index
# mkt_cap['Date'] = mkt_cap['Date'].apply(to_date)
mkt_cap['Date'] = pd.to_datetime(mkt_cap['Date'],format='%Y%m%d',errors='ignore')
mkt_cap = mkt_cap.set_index('Date')
mkt_cap = mkt_cap[['BTC','ETH','Crypto Market Cap']]
mkt_cap.head()

,BTC,ETH,Crypto Market Cap
Date,,,
2013-04-28,1.50388e+09,NaN,1.59617e+09
2013-05-05,1.26103e+09,NaN,1.34178e+09
2013-05-12,1.29353e+09,NaN,1.36792e+09
2013-05-19,1.37525e+09,NaN,1.44832e+09
2013-05-26,1.4779e+09,NaN,1.54898e+09


In [9]:
# https://trends.google.com/trends/explore?date=2013-04-28%202018-01-29&q=bitcoin,ethereum
df_goog = pd.read_csv('20180128_GoogleTrendsSearchInterest.csv') # Google Trends "bitcoin" interest over time
df_goog = df_goog.rename(columns={'Category: All categories': 'Date','Unnamed: 1':'Interest'})
df_goog['Date'] = pd.to_datetime(df_goog['Date'],format='%Y%m%d',errors='ignore')
df_goog = df_goog.set_index('Date')
df_goog = df_goog.iloc[2:]
df_goog.head()

,Interest
Date,
2013-04-28,3
2013-05-05,2
2013-05-12,2
2013-05-19,2
2013-05-26,2


In [10]:
df_mc = pd.concat([mkt_cap, df_goog], axis=1)
df_mc.tail()

,BTC,ETH,Crypto Market Cap,Interest
Date,,,,
2017-12-24,235595837210,66472347199,538097814846,63
2017-12-31,220903949498,69767510695,572573375760,48
2018-01-07,287582315011,106276577298,822534709108,47
2018-01-14,236679147035,132809805708,724968825266,58
2018-01-21,207286525403,107468021289,597208421609,38


In [11]:
df_all = pd.concat([df_hist, df_mc], axis=1)
df_all = df_all.fillna(method='ffill')
# start = int(datetime.datetime(2013,4,28))
df_all = df_all.iloc[200:,:]
df_all.head()

,BTCPrice,ETHPrice,BTCVol,TxFees,CostperTxn,NoTxns,NoperBlock,AvgBlkSz,UniqueAddresses,HashRate,Difficulty,Nasdaq,Gold,BTC,ETH,Crypto Market Cap,Interest
Date,,,,,,,,,,,,,,,,,
2016-07-13,656.669626,10.55,871225.091442,34693.70,5.69,219034.0,1531.70,0.77,379937.0,1516960.89,2.133989e+11,5005.73,227.19,1.02017e+10,8.88582e+08,1.22546e+10,3
2016-07-14,656.132060,10.49,872834.409470,35233.50,5.06,218093.0,1677.63,0.81,367252.0,1379055.35,2.133989e+11,5034.06,226.88,1.02017e+10,8.88582e+08,1.22546e+10,3
2016-07-15,663.841255,11.50,756558.628070,38613.99,5.49,220655.0,1553.90,0.79,367953.0,1506352.77,2.133989e+11,5029.59,224.00,1.02017e+10,8.88582e+08,1.22546e+10,3
2016-07-17,663.841255,11.50,756558.628070,38613.99,5.49,220655.0,1553.90,0.79,367953.0,1506352.77,2.133989e+11,5029.59,224.00,1.0473e+10,9.35944e+08,1.28827e+10,3
2016-07-18,680.149570,11.35,752528.346010,32557.41,7.59,171013.0,1147.73,0.70,330330.0,1580609.59,2.133989e+11,5055.78,224.43,1.0473e+10,9.35944e+08,1.28827e+10,3


In [12]:
df_all = df_all[['BTCPrice','ETHPrice','BTCVol','Crypto Market Cap', 'CostperTxn','TxFees','NoTxns','AvgBlkSz','UniqueAddresses','HashRate','Nasdaq','Gold','Interest']]
df_all = pd.DataFrame(df_all,dtype=np.float64)


In [13]:
df_all['logBTCPrice'] = df_all['BTCPrice'].apply(to_log)
df_all['logNasdaq'] = df_all['Nasdaq'].apply(to_log)
df_all['logETHPrice'] = df_all['ETHPrice'].apply(to_log)
df_all['logGold'] = df_all['Gold'].apply(to_log)
df_all['logCrypto Market Cap'] = df_all['Crypto Market Cap'].apply(to_log)

df_all.head()

,BTCPrice,ETHPrice,BTCVol,Crypto Market Cap,CostperTxn,TxFees,NoTxns,AvgBlkSz,UniqueAddresses,HashRate,Nasdaq,Gold,Interest,logBTCPrice,logNasdaq,logETHPrice,logGold,logCrypto Market Cap
Date,,,,,,,,,,,,,,,,,,
2016-07-13,656.669626,10.55,871225.091442,1.225459e+10,5.69,34693.70,219034.0,0.77,379937.0,1516960.89,5005.73,227.19,3.0,6.487181,8.518339,2.356126,5.425787,23.229166
2016-07-14,656.132060,10.49,872834.409470,1.225459e+10,5.06,35233.50,218093.0,0.81,367252.0,1379055.35,5034.06,226.88,3.0,6.486362,8.523982,2.350422,5.424421,23.229166
2016-07-15,663.841255,11.50,756558.628070,1.225459e+10,5.49,38613.99,220655.0,0.79,367953.0,1506352.77,5029.59,224.00,3.0,6.498043,8.523094,2.442347,5.411646,23.229166
2016-07-17,663.841255,11.50,756558.628070,1.288271e+10,5.49,38613.99,220655.0,0.79,367953.0,1506352.77,5029.59,224.00,3.0,6.498043,8.523094,2.442347,5.411646,23.279152
2016-07-18,680.149570,11.35,752528.346010,1.288271e+10,7.59,32557.41,171013.0,0.70,330330.0,1580609.59,5055.78,224.43,3.0,6.522313,8.528287,2.429218,5.413564,23.279152


In [14]:
df_all.to_pickle('benson_btcsentiment_df.pkl')

In [15]:
# Coinmarketcap: Current
url = 'https://coinmarketcap.com/all/views/all/'
response=requests.get(url)
page=response.text
soup=BeautifulSoup(page,"lxml")

tables=soup.find_all("table")

rows=[row for row in tables[0].find_all('tr')]
df = pd.read_html(tables[0].prettify())[0]
df.head()

,#,Name,Symbol,Market Cap,Price,Circulating Supply,Volume (24h),% 1h,% 24h,% 7d
0,1,BTC Bitcoin,BTC,"$189,462,036,224",$11254.70,16834037,"$7,058,720,000",-0.14%,-4.78%,4.81%
1,2,ETH Ethereum,ETH,"$114,601,284,982",$1177.91,97292055,"$3,713,550,000",-0.22%,-5.78%,19.56%
2,3,XRP Ripple,XRP,"$51,011,315,862",$1.32,"38,739,142,811 *","$1,112,830,000",-0.39%,-5.78%,3.37%
3,4,BCH Bitcoin Cash,BCH,"$27,920,512,493",$1648.30,16938975,"$366,221,000",-0.31%,-5.84%,2.84%
4,5,ADA Cardano,ADA,"$16,134,467,850",$0.622302,"25,927,070,538 *","$237,864,000",-0.42%,-6.77%,11.50%


In [16]:
# Coinmarketcap scraping: Bitcoin by time period
url = 'https://coinmarketcap.com/currencies/bitcoin/historical-data/?start=20130428&end=20180129'
response=requests.get(url)
page=response.text
soup=BeautifulSoup(page,"lxml")
tables=soup.find_all("table")

rows=[row for row in tables[0].find_all('tr')]
df = pd.read_html(tables[0].prettify())[0]
df['Date']=df['Date'].apply(to_date)
df = df.set_index('Date')
df.to_pickle('benson_btcsentiment_dfts.pkl')
df.head()

,Open,High,Low,Close,Volume,Market Cap
Date,,,,,,
2018-01-28,11475.3,12040.3,11475.3,11786.3,8350360000,193133000000
2018-01-27,11174.9,11614.9,10989.2,11440.7,7583270000,188054000000
2018-01-26,11256.0,11656.7,10470.3,11171.4,9746200000,189398000000
2018-01-25,11421.7,11785.7,11057.4,11259.4,8873170000,192163000000
2018-01-24,10903.4,11501.4,10639.8,11359.4,9940990000,183419000000


In [17]:
df.tail()

,Open,High,Low,Close,Volume,Market Cap
Date,,,,,,
2013-05-02,116.38,125.60,92.28,105.21,-,1292190000
2013-05-01,139.00,139.89,107.72,116.99,-,1542820000
2013-04-30,144.00,146.93,134.05,139.00,-,1597780000
2013-04-29,134.44,147.49,134.00,144.54,-,1491160000
2013-04-28,135.30,135.98,132.10,134.21,-,1500520000


In [18]:
# Coinmarketcap scraping: Ethereum
url = 'https://coinmarketcap.com/currencies/ethereum/historical-data/?start=20130428&end=20180129'
response=requests.get(url)
page=response.text
soup=BeautifulSoup(page,"lxml")
tables=soup.find_all("table")

rows=[row for row in tables[0].find_all('tr')]
df = pd.read_html(tables[0].prettify())[0]
df['Date']=df['Date'].apply(to_date)
df.to_pickle('benson_btcsentiment_dftseth.pkl')
print(df.head())

        Date     Open     High      Low    Close      Volume    Market Cap
0 2018-01-28  1111.78  1257.77  1111.78  1246.01  5372330000  108122000000
1 2018-01-27  1055.75  1121.98  1042.12  1107.07  3002870000  102651000000
2 2018-01-26  1052.70  1080.60   996.78  1055.17  3617690000  102334000000
3 2018-01-25  1063.22  1104.66  1034.74  1056.03  4050190000  103335000000
4 2018-01-24   987.48  1062.44   965.81  1058.78  3944040000   95952700000


In [19]:
btcsa = pd.read_json('btcpricesentiment_mc7_.json',convert_axes=False).T # connect to most recent active file

btcsa['Date'] = btcsa.index
btcsa['Date'] = pd.to_datetime(btcsa['Date'])
btcsa['Bitcoin_S'] = btcsa['bitcoin_S'].apply(list_to_average)
btcsa['Ethereum_S'] = btcsa['ethereum_S'].apply(list_to_average)
btcsa['Blockchain_S'] = btcsa['blockchain_S'].apply(list_to_average)
btcsa = btcsa[['Date','Bitcoin_S','Ethereum_S','Blockchain_S','BTC','ETH','LTC','ADA','EOS','Crypto Market Cap']]
btcsa = btcsa.sort_values('Date')
btcsa = btcsa.set_index('Date')
btcsa.to_pickle('benson_btcsentiment_dfs.pkl')
print(btcsa.shape)
btcsa.head(2)

(223, 9)


,Bitcoin_S,Ethereum_S,Blockchain_S,BTC,ETH,LTC,ADA,EOS,Crypto Market Cap
Date,,,,,,,,,
2018-01-28 18:13:00,0.218750,-0.173106,0.437179,"$193,540,756,417","$116,129,045,529","$10,144,386,235","$16,631,593,500","$9,254,599,532","$576,359,759,424"
2018-01-28 18:14:00,0.472064,0.472606,0.395227,"$193,540,756,417","$116,129,045,529","$10,144,386,235","$16,631,593,500","$9,254,599,532","$576,359,759,424"


In [20]:
# Bitcoin futures scraping
url = 'http://www.cmegroup.com/trading/equity-index/us-index/bitcoin.html'
response=requests.get(url)
page=response.text
soup=BeautifulSoup(page,"lxml")

tables=soup.find_all("table")

rows=[row for row in tables[0].find_all('tr')]
df = pd.read_html(tables[0].prettify())[0]
df = df[:5]
df

,Month,Charts,Last,Change,Prior Settle,Open,High,Low,Volume,Hi / Low Limit,Updated
0,FEB 2018,Show Price Chart,11245,+75,11170.0,11120,11245,11120,4.0,11950 / 10390,17:41:07 CT 29 Jan 2018
1,MAR 2018,Show Price Chart,-,-,11215.0,-,-,-,0.0,11970 / 10410,17:40:39 CT 29 Jan 2018
2,APR 2018,Show Price Chart,-,-,11255.0,-,-,-,0.0,11965 / 10405,17:42:49 CT 29 Jan 2018
3,JUN 2018,Show Price Chart,-,-,11235.0,-,-,-,0.0,11965 / 10405,16:45:00 CT 29 Jan 2018
4,Legend: Options Price Chart About This Report,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
# Resources
# https://marcobonzanini.com/2015/03/09/mining-twitter-data-with-python-part-2/
# http://cs229.stanford.edu/proj2015/029_report.pdf
# http://text-processing.com/ API
# https://trends.google.com/trends/explore?q=bitcoin,ethereum